
# Evaluation: Competitive Ratio (Batch)
Loop over offline/online solutions and compute competitive ratios for each pair.


In [1]:

from pathlib import Path
import sys
import pandas as pd
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().resolve().parents[1]))
from motion_planning.utils.paths import OFFLINE_RESULTS_DIR, ONLINE_RESULTS_DIR
from motion_planning.evaluation.competitive_ratio import compute_competitive_ratio

# Ensure repo root and src on path
cwd = Path.cwd().resolve()
REPO_ROOT = cwd
for parent in [cwd] + list(cwd.parents):
    if (parent / "src" / "motion_planning").exists():
        REPO_ROOT = parent
        break
src_path = REPO_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)


Repo root: /home/abdulrahman/competitive-online-algorithms-motion-planning



## Locate artifacts


In [2]:
from motion_planning.utils.paths import OFFLINE_RESULTS_DIR, ONLINE_RESULTS_DIR
from motion_planning.evaluation.competitive_ratio import compute_competitive_ratio
# Offline
offline_graph = sorted((OFFLINE_RESULTS_DIR / "solution details").glob("*_offline_graph_solution_edges.csv"))
offline_cand = sorted((OFFLINE_RESULTS_DIR / "problem details").glob("*_offline_candidates_opt.csv"))

# Online
online_graph = sorted((ONLINE_RESULTS_DIR / "graph-based").glob("*_online_ITM-ORB.csv"))
online_cand = sorted((ONLINE_RESULTS_DIR / "candidates").glob("*_online_*ORB.csv"))

print("Counts -> offline graph:", len(offline_graph), "offline cand:", len(offline_cand))
print("Counts -> online graph:", len(online_graph), "online cand:", len(online_cand))


Counts -> offline graph: 2 offline cand: 2
Counts -> online graph: 2 online cand: 4



## Competitive ratio for graph-based (utility)
Pair offline graph opt with online ITM-ORB by matching the base stem (before suffix).
CR = offline_utility / online_utility (utility objective).


In [3]:

# Build maps by base stem
offline_graph_map = {f.stem.replace("_offline_graph_solution_edges", ""): f for f in offline_graph}
online_graph_map = {f.stem.replace("_online_ITM-ORB", ""): f for f in online_graph}

records = []
for base, off_path in offline_graph_map.items():
    on_path = online_graph_map.get(base)
    if on_path is None:
        continue
    df_off = pd.read_csv(off_path)
    df_on = pd.read_csv(on_path)
    off_util = df_off.get("utility", pd.Series(dtype=float)).sum()
    on_util = df_on.get("utility", pd.Series(dtype=float)).sum()
    cr = compute_competitive_ratio(online_value=on_util, offline_value=off_util, objective="utility")
    records.append({
        "scenario": base,
        "offline_file": off_path.name,
        "online_file": on_path.name,
        "offline_utility": off_util,
        "online_utility": on_util,
        "cr_utility": cr,
    })

df_graph_cr = pd.DataFrame(records)
display(df_graph_cr)


,scenario,offline_file,online_file,offline_utility,online_utility,cr_utility
0,20251125_164202_Town03_Roundabout_10_mid_5,20251125_164202_Town03_Roundabout_10_mid_5_off...,20251125_164202_Town03_Roundabout_10_mid_5_onl...,1469.183927,1428.554274,1.028441
1,20251125_164202_Town03_Roundabout_15_mid_5,20251125_164202_Town03_Roundabout_15_mid_5_off...,20251125_164202_Town03_Roundabout_15_mid_5_onl...,1489.112909,1409.016311,1.056846



## Competitive ratio for candidates-based (utility)
Pair offline candidates opt with each online candidates run (CZL/BAT). For each match, compute CR = offline_utility / online_utility.


In [4]:

offline_cand_map = {f.stem.replace("_offline_candidates_opt", ""): f for f in offline_cand}
# group online by base stem
online_cand_entries = []
for f in online_cand:
    stem = f.stem
    if "_online_" in stem:
        base, alg = stem.split("_online_", 1)
    else:
        base, alg = stem, "unknown"
    online_cand_entries.append((base, alg, f))

records_cand = []
for base, off_path in offline_cand_map.items():
    df_off = pd.read_csv(off_path)
    off_util = df_off.get("utility", pd.Series(dtype=float)).sum()
    for b, alg, on_path in online_cand_entries:
        if b != base:
            continue
        df_on = pd.read_csv(on_path)
        on_util = df_on.get("utility", pd.Series(dtype=float)).sum()
        cr = compute_competitive_ratio(online_value=on_util, offline_value=off_util, objective="utility")
        records_cand.append({
            "scenario": base,
            "algorithm": alg,
            "offline_file": off_path.name,
            "online_file": on_path.name,
            "offline_utility": off_util,
            "online_utility": on_util,
            "cr_utility": cr,
        })

df_cand_cr = pd.DataFrame(records_cand)
display(df_cand_cr)


,scenario,algorithm,offline_file,online_file,offline_utility,online_utility,cr_utility
0,20251125_164202_Town03_Roundabout_10_mid_5,BAT-ORB,20251125_164202_Town03_Roundabout_10_mid_5_off...,20251125_164202_Town03_Roundabout_10_mid_5_onl...,1469.183927,1408.900945,1.042787
1,20251125_164202_Town03_Roundabout_10_mid_5,CZL-ORB,20251125_164202_Town03_Roundabout_10_mid_5_off...,20251125_164202_Town03_Roundabout_10_mid_5_onl...,1469.183927,1408.900945,1.042787
2,20251125_164202_Town03_Roundabout_15_mid_5,BAT-ORB,20251125_164202_Town03_Roundabout_15_mid_5_off...,20251125_164202_Town03_Roundabout_15_mid_5_onl...,1489.112909,1468.740443,1.013871
3,20251125_164202_Town03_Roundabout_15_mid_5,CZL-ORB,20251125_164202_Town03_Roundabout_15_mid_5_off...,20251125_164202_Town03_Roundabout_15_mid_5_onl...,1489.112909,1468.740443,1.013871
